In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -U langchain-community

In [ ]:
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import Together
import requests
import time
import json


In [ ]:
# ติดตั้ง PyTorch เวอร์ชันล่าสุด
!pip install torch==2.0.1

In [ ]:
# 🚀 ลบ PyTorch และ Transformers เก่าทั้งหมด
!pip uninstall -y torch torchvision torchaudio transformers

# 🚀 ติดตั้ง PyTorch ที่รองรับ CUDA ของ Colab (CUDA 11.8)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 🚀 ติดตั้ง Transformers เวอร์ชันที่รองรับ
!pip install transformers --upgrade


In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import json
import numpy as np

# 🚀 โหลดโมเดล BGE-M3 (BAAI)
model_name = "BAAI/bge-base-en"  # เปลี่ยนเป็นโมเดล BGE-M3
model = SentenceTransformer(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # ตั้งเป็นโหมดประเมินผลเพื่อประหยัดหน่วยความจำ

# 🚀 ฟังก์ชันสร้าง Embedding ของคำถาม
def embed_text_batch(texts):
    embeddings = model.encode(
        texts,
        convert_to_tensor=True,
        show_progress_bar=True,
        device=device,
        batch_size=32  # สามารถปรับ batch size ให้เหมาะสมกับ GPU ได้
    )
    return embeddings.cpu().numpy()

# 🚀 โหลด Evaluation Dataset
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_datasetv2.json'  # เปลี่ยนชื่อไฟล์เป็น v3
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 สร้าง Embeddings สำหรับคำถามทั้งหมด (Batch)
batch_size = 10  # แบ่งเป็น batch ละ 10 คำถาม
all_embeddings = []

for i in range(0, len(evaluation_data), batch_size):
    batch_questions = [item['question'] for item in evaluation_data[i:i + batch_size]]

    batch_embeddings = embed_text_batch(batch_questions)
    all_embeddings.extend(batch_embeddings.tolist())

    print(f"✅ batch ที่ {i // batch_size + 1} embed แล้ว")

    # 🚀 เคลียร์ Cache ของ GPU เพื่อประหยัดหน่วยความจำ
    if device.type == "cuda":
        torch.cuda.empty_cache()

# 🚀 เพิ่ม Embeddings กลับเข้า Evaluation Dataset
for i, item in enumerate(evaluation_data):
    item['embedding'] = all_embeddings[i]

print("✅ สร้าง Embeddings สำหรับคำถามใน Evaluation Dataset สำเร็จ")

# 🚀 กำหนดเส้นทางที่ต้องการบันทึกไฟล์ (Google Drive)
output_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v3_BGE-M3.json'  # เปลี่ยนเป็น v3

# 🚀 บันทึก Evaluation Dataset พร้อม Embeddings ลงไฟล์ JSON
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(evaluation_data, f, ensure_ascii=False, indent=4)

print(f"✅ บันทึก Evaluation Dataset พร้อม Embeddings ที่: {output_file}")


สร้าง vector database

In [ ]:
# 🚀 2. เชื่อมต่อกับ Chroma Database ที่มีอยู่แล้ว
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v3'  # ระบุ path ของฐานข้อมูล
client = chromadb.PersistentClient(path=db_path)

# โหลด Collection ที่มีอยู่แล้ว
collection = client.get_collection(name="recommendations")
print("✅ เชื่อมต่อกับ Chroma DB สำเร็จ")


K = 3

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings (ใช้ไฟล์ v3 BGE-M3)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v3_BGE-M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset พร้อม Embeddings จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB (ใช้ฐานข้อมูล v3)
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v3'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG ดึงคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(question_embedding, top_k=3):
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=top_k
    )
    if results and 'documents' in results and len(results['documents']) > 0:
        return results['documents'][0]
    return []

print("✅ พร้อมสำหรับการค้นหาคำแนะนำด้วย RAG")

# 🚀 ทดลองดึงคำแนะนำที่ใกล้เคียงสำหรับคำถามแรก
test_question = evaluation_data[0]['question']
test_embedding = evaluation_data[0]['embedding']

recommendations = retrieve_recommendations(test_embedding)
print(f"\n🔎 คำถาม: {test_question}\n✅ คำแนะนำที่ใกล้เคียง:\n")

for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec}")


In [2]:
import json

# 🚀 โหลดไฟล์ evaluation dataset (คำถาม-คำตอบจริง)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v3_BGE-M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# 🚀 โหลดไฟล์ผลลัพธ์ RAG (คำถาม + คำแนะนำที่ดึงมา)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v3_BGE-M3_K=3.json'
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 3  # จำนวนคำแนะนำที่ดึงมา (top_k)

# 🚀 ฟังก์ชันคำนวณ Precision@K
def precision_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / k if k > 0 else 0

# 🚀 ฟังก์ชันคำนวณ Recall@K
def recall_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / len(relevant) if relevant else 0

# 🚀 ฟังก์ชันคำนวณ Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(relevant, retrieved):
    for idx, rec in enumerate(retrieved, 1):
        if rec in relevant:
            return 1 / idx
    return 0

# 🚀 เก็บผลลัพธ์คำนวณต่างๆ
precisions = []
recalls = []
mrrs = []

print("🔄 กำลังคำนวณ Precision, Recall, และ MRR...")

for idx, item in enumerate(rag_results, 1):
    # ตรวจสอบว่ามีคำตอบและคำแนะนำหรือไม่
    if 'answer' not in item or 'recommendations' not in item:
        print(f"⚠️ ข้อที่ {idx} ไม่มีคำตอบหรือคำแนะนำ ข้ามไป...")
        continue

    # คำตอบจริง (ในรูปแบบ list เพื่อรองรับหลายคำตอบในอนาคต)
    relevant_answers = [item['answer']] if isinstance(item['answer'], str) else item['answer']

    # คำแนะนำที่ดึงมา (list of strings)
    retrieved = item['recommendations']

    # คำนวณ Metrics
    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

    if idx % 100 == 0 or idx == len(rag_results):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(rag_results)} คำถาม")

# 🚀 คำนวณค่าเฉลี่ยของ Precision, Recall, MRR ทั้งหมด
avg_precision = sum(precisions) / len(precisions) if precisions else 0
avg_recall = sum(recalls) / len(recalls) if recalls else 0
avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0

print("\n🔎 ผลลัพธ์การประเมินผลด้วย RAG")
print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


🔄 กำลังคำนวณ Precision, Recall, และ MRR...
✅ ประมวลผลแล้ว 100/1370 คำถาม
✅ ประมวลผลแล้ว 200/1370 คำถาม
✅ ประมวลผลแล้ว 300/1370 คำถาม
✅ ประมวลผลแล้ว 400/1370 คำถาม
✅ ประมวลผลแล้ว 500/1370 คำถาม
✅ ประมวลผลแล้ว 600/1370 คำถาม
✅ ประมวลผลแล้ว 700/1370 คำถาม
✅ ประมวลผลแล้ว 800/1370 คำถาม
✅ ประมวลผลแล้ว 900/1370 คำถาม
✅ ประมวลผลแล้ว 1000/1370 คำถาม
✅ ประมวลผลแล้ว 1100/1370 คำถาม
✅ ประมวลผลแล้ว 1200/1370 คำถาม
✅ ประมวลผลแล้ว 1300/1370 คำถาม
✅ ประมวลผลแล้ว 1370/1370 คำถาม

🔎 ผลลัพธ์การประเมินผลด้วย RAG
Precision@3: 0.001
Recall@3: 0.003
Mean Reciprocal Rank (MRR): 0.002


K = 5

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v3_BGE-M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset พร้อม Embeddings จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v3'  # เปลี่ยนเป็น Database ใหม่
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG (Retrieve) เพื่อค้นหาคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(query_embedding, top_k=5):
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0] if results['documents'] else []

# 🚀 ใช้ RAG ค้นหาคำแนะนำสำหรับทุกคำถามใน Dataset
rag_results = []

print("🔄 กำลังดึงคำแนะนำที่ใกล้เคียงด้วย RAG...")

for idx, item in enumerate(evaluation_data, 1):
    query_embedding = item['embedding']
    recommendations = retrieve_recommendations(query_embedding, top_k=5)

    rag_results.append({
        "question": item['question'],
        "answer": item['answer'],
        "category": item['category'],
        "recommendations": recommendations
    })

    if idx % 100 == 0 or idx == len(evaluation_data):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(evaluation_data)} คำถาม")

# 🚀 บันทึกผลลัพธ์ RAG
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v3_BGE-M3_K=5.json'
with open(rag_results_file, 'w', encoding='utf-8') as f:
    json.dump(rag_results, f, ensure_ascii=False, indent=4)

print(f'✅ บันทึกผลลัพธ์ RAG ที่: {rag_results_file}')


In [3]:
import json

# 🚀 โหลดไฟล์ evaluation dataset (คำถาม-คำตอบจริง)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v3_BGE-M3.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# 🚀 โหลดไฟล์ผลลัพธ์ RAG (คำถาม + คำแนะนำที่ดึงมา)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_v3_BGE-M3_K=5.json'
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 5  # จำนวนคำแนะนำที่ดึงมา (top_k)

# 🚀 ฟังก์ชันคำนวณ Precision@K
def precision_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / k if k > 0 else 0

# 🚀 ฟังก์ชันคำนวณ Recall@K
def recall_at_k(relevant, retrieved, k):
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / len(relevant) if relevant else 0

# 🚀 ฟังก์ชันคำนวณ Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(relevant, retrieved):
    for idx, rec in enumerate(retrieved, 1):
        if rec in relevant:
            return 1 / idx
    return 0

# 🚀 เก็บผลลัพธ์คำนวณต่างๆ
precisions = []
recalls = []
mrrs = []

print("🔄 กำลังคำนวณ Precision, Recall, และ MRR...")

for idx, item in enumerate(rag_results, 1):
    # ตรวจสอบว่ามีคำตอบและคำแนะนำหรือไม่
    if 'answer' not in item or 'recommendations' not in item:
        print(f"⚠️ ข้อที่ {idx} ไม่มีคำตอบหรือคำแนะนำ ข้ามไป...")
        continue

    # คำตอบจริง (ในรูปแบบ list เพื่อรองรับหลายคำตอบในอนาคต)
    relevant_answers = [item['answer']] if isinstance(item['answer'], str) else item['answer']

    # คำแนะนำที่ดึงมา (list of strings)
    retrieved = item['recommendations']

    # คำนวณ Metrics
    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

    if idx % 100 == 0 or idx == len(rag_results):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(rag_results)} คำถาม")

# 🚀 คำนวณค่าเฉลี่ยของ Precision, Recall, MRR ทั้งหมด
avg_precision = sum(precisions) / len(precisions) if precisions else 0
avg_recall = sum(recalls) / len(recalls) if recalls else 0
avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0

print("\n🔎 ผลลัพธ์การประเมินผลด้วย RAG")
print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


🔄 กำลังคำนวณ Precision, Recall, และ MRR...
✅ ประมวลผลแล้ว 100/1370 คำถาม
✅ ประมวลผลแล้ว 200/1370 คำถาม
✅ ประมวลผลแล้ว 300/1370 คำถาม
✅ ประมวลผลแล้ว 400/1370 คำถาม
✅ ประมวลผลแล้ว 500/1370 คำถาม
✅ ประมวลผลแล้ว 600/1370 คำถาม
✅ ประมวลผลแล้ว 700/1370 คำถาม
✅ ประมวลผลแล้ว 800/1370 คำถาม
✅ ประมวลผลแล้ว 900/1370 คำถาม
✅ ประมวลผลแล้ว 1000/1370 คำถาม
✅ ประมวลผลแล้ว 1100/1370 คำถาม
✅ ประมวลผลแล้ว 1200/1370 คำถาม
✅ ประมวลผลแล้ว 1300/1370 คำถาม
✅ ประมวลผลแล้ว 1370/1370 คำถาม

🔎 ผลลัพธ์การประเมินผลด้วย RAG
Precision@5: 0.001
Recall@5: 0.003
Mean Reciprocal Rank (MRR): 0.002
